In [5]:
import lxml
from lxml.builder import ElementMaker
from ncclient import manager

BASE_NS_1_0 = "urn:ietf:params:xml:ns:netconf:base:1.0"
H3C_BASE_1_0 = "http://www.h3c.com/netconf/base:1.0"
H3C_CONFIG_1_0 = "http://www.h3c.com/netconf/config:1.0"
H3C_DATA_1_0 = "http://www.h3c.com/netconf/data:1.0"
H3C_DATA_1_0_C = '{' + H3C_DATA_1_0 + '}'
H3C_ACTION_1_0 = "http://www.h3c.com/netconf/action:1.0"

host = {
    'host': '192.168.56.20',
    'username': 'netdevops',
    'password': 'netdevops',
    'port': 830,
    'device_params':{'name': 'h3c'},
}
conn = manager.connect(**host,hostkey_verify=False,look_for_keys=False)

In [ ]:
iface_key_map = {
    '接口名称': 'Name',
    'MTU': 'ActualMTU',
    'IP 地址':'InetAddressIPV4',
}
iface_key_map.get('Name',{}).get('111222','111')

对设备进行配置锁定，防止同时操作出现问题：

In [ ]:
# print(conn.lock(target='running'))
print(conn.unlock('running'))

返回值为以下内容：
```xml
<?xml version="1.0" encoding="UTF-8"?><rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:a5cf71cd-8505-4740-8165-dbff76489147"><ok/></rpc-reply>
```
再次登录到设备，无法进行配置操作。
```
[R1]int g0/1
The system has been locked by NETCONF. Please try later.
```

In [6]:
from lxml import etree
# H3C 自有部分默认情况下都以 top 元素为起点
E = ElementMaker(namespace=H3C_DATA_1_0, nsmap={None: H3C_DATA_1_0})
get_iface = E.top(
    E.Ifmgr(
        E.Interfaces(
            E.Interface(
                E.Name()
            )
        )
    )
)
print(etree.tostring(get_iface))

b'<top xmlns="http://www.h3c.com/netconf/data:1.0"><Ifmgr><Interfaces><Interface><Name/></Interface></Interfaces></Ifmgr></top>'


```xml
b'<top xmlns="http://www.h3c.com/netconf/data:1.0"><Ifmgr><Interfaces><Interface><Name/></Interface></Interfaces></Ifmgr></top>'
```

In [23]:
ret = conn.get(('subtree', get_iface))
print(dir(ret))

['ERROR_CLS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_data', '_errors', '_huge_tree', '_parsed', '_parsing_hook', '_raw', '_root', 'data', 'data_ele', 'data_xml', 'error', 'errors', 'ok', 'parse', 'xml']


In [27]:
print(dir(ret.data_ele))

['__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '_init', 'addnext', 'addprevious', 'append', 'attrib', 'base', 'clear', 'cssselect', 'extend', 'find', 'findall', 'findtext', 'get', 'getchildren', 'getiterator', 'getnext', 'getparent', 'getprevious', 'getroottree', 'index', 'insert', 'items', 'iter', 'iterancestors', 'iterchildren', 'iterdescendants', 'iterfind', 'itersiblings', 'itertext', 'keys', 'makeelement', 'nsmap', 'prefix', 'remove', 'replace', 'set', 'sourceline', 'tag', 'tail', 'text', 'values', 'xpath']


In [66]:
a = ret.data_ele

In [86]:
a.findall("Name")

[]

```xml
<?xml version="1.0" encoding="UTF-8"?><rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:7638826e-af96-4542-a541-3430ea08d232"><data><top xmlns="http://www.h3c.com/netconf/data:1.0"><Ifmgr><Interfaces><Interface><IfIndex>1</IfIndex><Name>GigabitEthernet0/0</Name></Interface><Interface><IfIndex>2</IfIndex><Name>GigabitEthernet0/1</Name></Interface><Interface><IfIndex>3</IfIndex><Name>GigabitEthernet0/2</Name></Interface><Interface><IfIndex>4</IfIndex><Name>Serial1/0</Name></Interface><Interface><IfIndex>5</IfIndex><Name>Serial2/0</Name></Interface><Interface><IfIndex>6</IfIndex><Name>Serial3/0</Name></Interface><Interface><IfIndex>7</IfIndex><Name>Serial4/0</Name></Interface><Interface><IfIndex>8</IfIndex><Name>GigabitEthernet5/0</Name></Interface><Interface><IfIndex>9</IfIndex><Name>GigabitEthernet5/1</Name></Interface><Interface><IfIndex>10</IfIndex><Name>GigabitEthernet6/0</Name></Interface><Interface><IfIndex>11</IfIndex><Name>GigabitEthernet6/1</Name></Interface><Interface><IfIndex>129</IfIndex><Name>NULL0</Name></Interface><Interface><IfIndex>130</IfIndex><Name>InLoopBack0</Name></Interface><Interface><IfIndex>131</IfIndex><Name>Register-Tunnel0</Name></Interface></Interfaces></Ifmgr></top></data></rpc-reply>
```

In [8]:
# 由于只能通过 IfIndex 来配置接口，所以要先根据接口名称来获取到 IfIndex
# TODO: 可以做获取所有接口信息的函数并存入数据库，实际操作的时候，可以让用户直接下拉选择；
# 每次操作的时候获取一次最新数据，如果不操作，则定期更新
def _get_iface_index(iface_name, conn):
    """通过接口名称来获取接口索引"""
    E = ElementMaker(namespace=H3C_DATA_1_0, nsmap={None: H3C_DATA_1_0})
    get_iface = E.top(
        E.Ifmgr(
            E.Interfaces(
                E.Interface(
                    E.Name(iface_name)
                )
            )
        )
    )
    nc_get_reply = conn.get(('subtree', get_iface))
    reply_data = nc_get_reply.data_ele.find('.//{}{}'.format(H3C_DATA_1_0_C, 'IfIndex'))
    if reply_data is None:
        return ''
    return reply_data.text

# ret = _get_iface_index('GigabitEthernet0/1',conn)
# print(ret)

In [11]:
# 获取到接口索引后进行下发接口配置
iface = 'GigabitEthernet0/1'
desc = 'Configured by netconf111'
E = ElementMaker(namespace=H3C_CONFIG_1_0, nsmap={None: H3C_CONFIG_1_0})
C = ElementMaker(namespace=BASE_NS_1_0, nsmap = {None: BASE_NS_1_0})
config = C.config(
    E.top(
    E.Ifmgr(
        E.Interfaces(
            E.Interface(
                E.IfIndex(_get_iface_index(iface,conn)),
                E.Description(desc)
            )
        )
    )
))

ret = conn.edit_config(target='running',config=config)
print(ret)

# 追加xml的方法：find 到对应的 tag，使用 append 方法
# 例如：find_in_action('Interface', top).append(E.Remove())

<?xml version="1.0" encoding="UTF-8"?><rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:623a5b0e-2d40-4bec-8ca0-6d629415e456"><ok/></rpc-reply>


In [ ]:
from pyhpecw7.comware import HPCOM7
from pyhpecw7.features.interface import Interface
conn1 = HPCOM7(**host)
conn1.open()

In [ ]:
interface = Interface(conn1, 'gi0/2')
iface_config = {
    'admin': '1',
    'description': 'configured by hpcmw7',
}
interface.build(**iface_config)
interface.update()
interface.get_config()

In [ ]:
#  不能拆分子接口，修改代码
interface._get_number('Gi1/1.10:100')

In [ ]:
def reverse_value_map(key_map, value_map):
    """Utility function for creating a
    "reverse" value map from a given key map and value map.
    """
    r_value_map = {}
    for k, v in value_map.items():
        sub_values = r_value_map[key_map[k]] = {}
        for k2, v2 in v.items():
            sub_values[v2] = k2

    return r_value_map

_key_map = {
            'admin': 'AdminStatus',
            'speed': 'ConfigSpeed',
            'duplex': 'ConfigDuplex',
            'description': 'Description',
            'type': 'PortLayer'
        }
_value_map = {
            'AdminStatus': {'1': 'up',
                            '2': 'down'},
            'ConfigSpeed': {'1': 'auto', '2': '10',
                            '4': '100', '32': '1000',
                            '1024': '10000', '4096': '20000',
                            '8192': '40000', '16384': '100000'},
            'ConfigDuplex': {'1': 'full',
                             '2': 'half',
                             '3': 'auto'},
            'PortLayer': {'1': 'bridged',
                          '2': 'routed'}
        }
_r_key_map = dict(reversed(item) for item in _key_map.items())
print(_r_key_map)
_r_value_map = reverse_value_map(_r_key_map, _value_map)
print(_r_value_map)

In [ ]:
class A():
    def __init__(self, a = None):
        if a:
            self.a = a
    def func(self, a):
        dic['a'] = self.a
        print(a)

a = '111'
aa = A()
dir(aa.func(a))